In [1]:
from solid import *
from solid.utils import *

from subprocess import run

import viewscad
r = viewscad.Renderer()

In [2]:
import os, solid; print(os.path.dirname(solid.__file__) + '/examples')

/home/flo/.local/lib/python3.9/site-packages/solid/examples


In [38]:
d = cube([7,4,4]) + sphere(5) - cylinder(r=2, h=6)
r.render(d)

In [8]:
projection(d) # muss ich vlt doch in openscad machen

In [45]:
# generate valid openscad code and store it in file
scad_render_to_file(d, 'd.scad')

# run openscad and export to stl
run(["openscad", "-o",  "d.stl", "d.scad"])

CompletedProcess(args=['openscad', '-o', 'd.stl', 'd.scad'], returncode=0)

In [46]:
def keyswitch(withCap=False):
    switch = cube([10,10,1])
    switch -= right(1)(forward(1)(cube([3,3,1])))
    if(withCap):
        cap = color([0.0,1.0,0.5])(cube([5,5,3]))
        cap = up(5)(cap)
        switch += cap
    switch = left(5)(back(5)(switch))
    return switch

s = keyswitch(True)
r.render(s)

In [44]:
s = keyswitch(True)
r.render(s)